`dict`类型不仅广泛使用，而且是Python语言的基石。模块的命名空间、实例的属性和函数的关键字参数中都可以看到字典的身影，可以用`__builtins__.__dict__`来查看字典相关的内置函数。\
因为字典很重要，Python专门针对字典进行了优化——散列表是字典类型性能出众的根本原因。

## 3.1 泛映射类型
`collections.abc`模块中有`Mapping`和`MutableMapping`两个抽象基类，主要作用是作为形式化的文档，定义了构建一个映射类型所需要的最基本的接口。非抽象映射类型一般不会直接继承这些抽象基类，而是直接对`dict`或者`collections.UserDict`进行扩展。

标准库里所有映射类型都是利用`dict`来实现的，***它们的共同限制是只有可散列的数据类型才能作为这些映射的键***。

In [19]:
from collections import abc
my_dict = {}
isinstance(my_dict, abc.Mapping) # 判断某个数据是不是广义的映射类型

True

> 可散列对象在其生命周期中，其散列值不变，且需要实现`__hash__`方法。另外可散列对象还要有`__eq__`方法，这样才能和其他键做比较。如果两个可散列对象相等，其散列值一定一样。

> 原子不可变数据类型都是可散列数据类型，比如`str`, `bytes`和`数值类型`，`frozenset`也是可散列的。只有当元组中包含的所有元素都是可散列类型的情况下，元组才是可散列的。

In [20]:
tt = (1, 2, (30,40))
hash(tt)

8027212646858338501

In [21]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [22]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

985328935373711578

> 一般用户自定义的类型的对象都是可散列的，散列值是它们`id`函数的返回值，所以所有这些对象在比较时都是不相等的；如果一个对象实现了`__eq__`，且该方法用到了该对象的内部状态，那么只有当这些内部状态不可变的情况下，该对象才是可散列的。

In [14]:
# 字典的多种定义方法
# a = dict('one'=1, 'two'=2, 'three'=3) 错
a = dict(one=1, two=2, three=3)
b = {'one':1, 'two':2, 'three':3}
c = dict(zip(['one', 'two', 'three'], [1,2,3]))
d = dict([('one', 1), ('two', 2), ('three', 3)])
e = dict({'three':3, 'two':2, 'one':1})
a == b == c == d == e

True

## 3.2 字典推导`dict comprehension`
`dictcomp`可以从任何以键值对作为元素的可迭代对象中构造出字典。

In [8]:
# 字典推导的应用
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia')
]

country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62}

In [13]:
code_country = {code: country.upper() for country, code in country_code.items() if code>=65}
code_country

{86: 'CHINA', 91: 'INDIA'}

## 3.3 常见的映射方法
`dict`, `collections.defaultdict` and `collections.OrderedDict`的常见方法：
* `defaultdict`: `d.default_factory`，在`__missing__`函数中调用，用来给未找到的元素设置值，这不是一个方法，而是一个可调用对象，它的值在`defaultdict`初始化的时候由用户给出；
* `d[k]`调用`d.__getitem__(k)`方法，如果找不到对应键时，`d.__missing__(k)`方法会被调用；
* `OrderedDict.popitem`会移除最后加入的元素（后进先出），若其可选`last`参数设置为False，则遵循先进先出（与书中不一致）;

* `d.setdefault(k, [default])`: 如果字典中有键k，返回其对应的值，否则让`d[k]=default`，然后返回`default`；
* `d.update(m, [**kargs])`方法，`m`可以是映射或键值对迭代器（“鸭子类型”，先检查m参数是否有`keys`方法，如果有，把`m`当作映射对象，否则当作包含了键值对的的迭代器）。

In [68]:
from collections import OrderedDict
d = OrderedDict()
d['one'] = 1
d['two'] = 2
d['three'] = 3
d

OrderedDict([('one', 1), ('two', 2), ('three', 3)])

In [32]:
d.popitem()

('three', 3)

### 用`setdefault`处理找不到的键
`d[k]`找不到键的时候，Python会抛出异常，尽管可以用`d.get(k, default)`来代替，但是不方便更新键对应的值。

In [43]:
# sys.argv[1] 获取的是下列命令中的zen.txt
# python3 run.py zen.txt

import re
WORD_RE = re.compile(r'\w+')  # 匹配任意单词

index = {}
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1): # line_no从1开始计数
        for match in WORD_RE.finditer(line): # 该行匹配的所有结果（即全部单词）返回一个迭代器，循环读取
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
#             # 写法1: 以下写法不好
#             occurrences = index.get(word, [])
#             occurrences.append(location)
#             index[word] = occurrences # 把新列表放回字典，又涉及一次查询操作

            # 写法2: 使用setdefault
            index.setdefault(word, []).append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])
    break

a [(19, 48), (20, 53)]


```
my_dict.setdefault(key, []).append(new_value) # 1次键查询
```
等同于

```
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value) # 至少2次键查询，如果键不存在，则是3次
```

## 3.4 映射的弹性键查询
如果是单纯地查找取值，而不是通过查找插入新值，又该如何处理呢？如果我们希望映射中不存在的键有读取的默认值，我们可以通过
* `defaultdict`类
* 或 自定义`dict`的一个子类，然后在子类中实现`__missing__`方法

来实现。
### 3.4.1 `defaultdict`：处理找不到的键的一个选择
在实例化`defaultdict`的时候，需要给构造方法提供一个可调用对象，这个可调用对象会在`__getitem__`找不到键的时候被调用。比如新建字典——
```
dd = defaultdict(list)
```
如果键`'new-key'`不在`dd`中，则表达式`dd['new-key']`会按照如下步骤行事：
1. 调用`list()`生成一个新列表；
2. 把新列表作为值，`'new-key'`作为键，放到`dd`中；
3. 返回这个列表的引用。

这个用来生成默认值的可调用对象存放在名为`default_factory`的实例属性中。
> `default_factory`只会在`__getitem__`中被调用，而在`dd.get(k)`中不会发挥作用。因为如下节所说，`__missing__`方法只会被`__getitem__`调用，对`get`或`__contains__`无效。

背后功臣是`__missing__`特殊方法，这个特性是所有映射类型都可以选择去支持的，下一节会尝试为自定义`dict`类实现`__mising__`方法。

In [47]:
import collections
index = collections.defaultdict(list)
index.default_factory

list

In [46]:
import re
import sys
import collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict(list) # 
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            
            index[word].append(location) # 
        
for word in sorted(index, key=str.upper):
    print(word, index[word])
    break

a [(19, 48), (20, 53)]


### 3.4.2 特殊方法`__missing__`
虽然基类`dict`没有实现该方法，但是`dict`是知道这个东西存在的，继承自`dict`的某个类如果提供了`__missing__`方法，也会在`__getitem__`碰到找不到的键时调用它。

In [60]:
# StrKeyDict0在查询时把非字符串的键转换为字符串
class StrKeyDict0(dict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default):
        try:
            return self[key]
        except KeyError: # 如果抛出KeyError，说明__missing__也失败了，于是返回default
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() # 不能使用key in my_dict，否则会陷入无限递归


In [64]:
my_dict = StrKeyDict0([('2', 'two'), ('1', 'one')])
print(my_dict)

print(my_dict['1'])
print(my_dict[2])
# print(my_dict[3]) # KeyError
print(my_dict.get(3, 'None'))


{'2': 'two', '1': 'one'}
one
two
None


In [66]:
2 in my_dict

True

>诸如`k in my_dict.keys()`一类的操作在Python3中很快，即使映射类型对象很大也没关系，因为`dict.keys()`的返回值是一个视图，视图就像一个集合，和字典一样在查找元素时速度很快；而Python2里的`dict.keys()`返回的是一个列表，在处理体积较大的对象时候效率不高，因为`k in my_list`需要扫描整个列表。

以上是`dict`和`defaultdict`，下面看标准库里其他的映射类型。
## 3.5 字典的变种
```
collections.OrderedDict
```
添加键的时候会保持顺序，`popitem`方法默认删除最后一个元素（先进先出）。
```
collections.ChainMap
```
可以容纳数个不同的映射对象，然后在进行键查找操作的时候，这些对象会被当做一个整体被逐个查找，直到键被找到。这个功能在给有嵌套作用域的语言做解释器的时候很有用，可以用一个映射对象来代表一个作用域的上下文。

In [74]:
import builtins
pylookup = collections.ChainMap(locals(), globals(), vars(builtins)) # 是当前notebook的

```
collections.Counter
```
这个映射类型会给键准备一个整数计数器，每次更新一个键的时候都会增加这个计数器。
1. 给可散列对象计数
2. 当成多重集来用，即集合里的元素可以出现不止一次

`most_common([n])`返回映射中最常见的n个键和它们的计数。

In [75]:
ct = collections.Counter('abracdareab')
ct

Counter({'a': 4, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'e': 1})

In [76]:
ct.update('aaaaaaz')
ct

Counter({'a': 10, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'e': 1, 'z': 1})

In [78]:
ct.most_common()

[('a', 10), ('b', 2), ('r', 2), ('c', 1), ('d', 1), ('e', 1), ('z', 1)]

In [79]:
ct.most_common(2)

[('a', 10), ('b', 2)]

```
collections.UserDict
```
这个类其实就是把标准dict用Python实现了一遍，是为了让用户继承写子类的。

## 3.6 子类化`UserDict`
以`UserDict`为基类创建自定义映射类型比以普通的`dict`方便，主要原因是后者有些方法的实现会走捷径，导致我们不得不在子类中重写这些方法。

`UserDict`不是`dict`的子类，但是它有个属性`data`是`UserDict`的实例，这个属性实际是`UserDict`存储数据的地方。

In [80]:
# 无论是添加、更新还是查询操作，StrKeyDict都会把非字符串的键转换为字符串
import collections

class StrKeyDict(collections.UserDict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

In [81]:
my_dict = StrKeyDict([(1, 'one'), (2, 'two')])
my_dict

{'1': 'one', '2': 'two'}

In [82]:
my_dict[1]

'one'

In [83]:
my_dict['2']

'two'

In [84]:
my_dict[3] = 'three'
my_dict['3']

'three'

因为`UserDict`继承的是`MutableMapping`，所以`StrKeyDict`里剩下的那些映射类型的方法都是从`UserDict`，`MutableMapping`和`Mapping`这些超类继承来的。`Mapping`虽然是ABC，但提供了好几个实用方法。
```
MutableMapping.update
```
这个方法不但可以直接利用，还用在`__init__`里，让构造方法用传入的各种参数新建实例。这个方法背后使用`self[key] = value`来添加新值的，所以其实是在用`__setitem__`方法。
```
Mapping.get
```
`StrKeyDict`继承了`Mapping.get`方法，这跟我们写的`StrKeyDict0`中的`get`方法一样，所以不用重新实现以使其与`__getitem__`一致了。

## 3.7 不可变映射类型
标准库中的映射类型都是可变的。

从Python3.3开始，`types`模块中引入了一个封装类名叫`MappingProxyType`，如果给这个类一个映射，它会返回一个动态的只读视图，即原映射修改了，视图会变，但不能通过视图修改原映射。

In [85]:
# 用MappingProxyType来获取字典的只读实例mappingproxy
from types import MappingProxyType

d = {'1': 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({'1': 'A'})

In [86]:
d_proxy['1']

'A'

In [87]:
d_proxy['2'] = 'B'

TypeError: 'mappingproxy' object does not support item assignment

In [88]:
d['2'] = 'B'
d_proxy

mappingproxy({'1': 'A', '2': 'B'})

以上是标准库中的映射类型，下面是集合类型。
## 3.8 集合论
`set`和`frozenset`在Python2.6中才升级为内置类型。
集合本质是许多唯一对象的聚集。

In [89]:
# 集合用于去重
l = ['spam', 'eggs', 'spam', 'spam']
set(l)

{'eggs', 'spam'}

集合中的元素本身必须是可散列的，`set`类型本身不可散列，但是`frozenset`可以。因此可以创建一个包含`frozenset`的`set`。

集合提供了很多中缀运算符。

In [97]:
a = set((1, 2, 3, 4, 5))
b = set((4, 5, 6, 7))
print(a | b) # 合集
print(a & b) # 交集
print(a - b) # 差集

{1, 2, 3, 4, 5, 6, 7}
{4, 5}
{1, 2, 3}


In [99]:
a = set((1, 2, 3, 4, 5))
b = set((4, 5, 6, 7))
# 下例显示了中缀运算符的使用可以省去不必要的循环和逻辑操作
found = len(a & b)
print(found)

# 不使用中缀运算符，优点是可以用在任何迭代类型上，缺点是慢
found = 0
for n in b:
    if n in a:
        found += 1
print(found)

# 但我们可以随时创建set类型
a = [1, 2, 3, 4, 5]
b = [4, 5, 6, 7]
found = len(set(a).intersection(b))
print(found)

2
2
2


### 3.8.1 集合字面量
除空集外，集合的字面量`{1}, {1, 2}`等和它的数学形式一样，空集必须写成set()的形式。

In [101]:
a = {}
type(a)

dict

In [103]:
a

{}

In [102]:
b = set()
type(b)

set

In [104]:
b

set()

像`{1, 2, 3}`这种字面量句法比`set([1, 2, 3])`要更快更易读，因为Python必须从set这个名字查询构造方法，然后新建一个列表，最后再把列表传入构造方法。对于前者，Python会用一个专门的叫作BUILD_SET的字节码来创建集合。

In [106]:
from dis import dis
# dis.dis 反汇编函数

In [107]:
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [108]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


Python里没有针对frozenset的特殊字面量句法，所以只能用构造函数。

In [109]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

### 3.8.2 集合推导

In [116]:
import unicodedata

In [118]:
print(unicodedata.name('a'))
print(unicodedata.lookup(unicodedata.name('H')))

LATIN SMALL LETTER A
H


In [135]:
# 新建一个Latin-1字符集合，该集合中每个字符的Unicode名字里都含有‘SIGN这个单词
print({chr(i) for i in range(32,256) if 'SIGN' in unicodedata.name(chr(i), '')})

{'#', '£', '÷', '®', '¢', '×', '°', '+', '±', '¤', '¥', 'µ', '$', '<', '>', '¬', '¶', '%', '©', '=', '§'}


In [136]:
unicodedata.name('#')

'NUMBER SIGN'

### 3.8.3 集合的操作

集合操作中的中缀运算符要求两侧的操作对象都是集合类型，但是其他的所有方法则只要求传入的参数是可迭代对象。

In [171]:
a, b, c, d = set(range(0, 3)), list(range(3, 5)), list(range(5, 8)), tuple(range(8, 11))
e = a.union(b, c, d)
e

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [172]:
a = set(range(1, 5))
b = set(range(3, 7))
a, b

({1, 2, 3, 4}, {3, 4, 5, 6})

In [174]:
a ^ b # 对称差集，a和b中不属于a交b的元素的集合

{1, 2, 5, 6}

In [176]:
a.isdisjoint(b) # 相交判断

False

In [178]:
1 in a # contain与否

True

In [179]:
{1, 2} <= a # 子集判断 s.__le__(z)

True

In [180]:
{1, 2}.issubset(a) # 子集判断

True

In [181]:
{1, 2} < a # 真子集判断 s.__lt__(z)

True

In [182]:
a > {1, 2} and a >= {1, 2} # 父集判断

True

集合的其他操作

In [203]:
a = set(range(0, 3))
a

{0, 1, 2}

In [204]:
a.add(11)
a

{0, 1, 2, 11}

In [205]:
b = a.copy() # 浅拷贝
print(a, b)
a.add(23)
print(a, b)

{0, 1, 2, 11} {0, 1, 2, 11}
{0, 1, 2, 11, 23} {0, 1, 2, 11}


In [207]:
a.discard(23)
a

{0, 1, 2, 11}

In [208]:
a.discard(45) # 如果a中有该元素，则移除，否则并不报错
# a.remove(45) # 会报错，KeyError
a

{0, 1, 2, 11}

In [209]:
a.pop()

0

In [210]:
print(a)
a.clear()
print(a)

{1, 2, 11}
set()


## 3.9 `dict`和`set`背后
* Python里dict和set的效率有多高？
* 为什么它们是无序的？
* 为什们不是所有Python对象都可以当作dict的键或set的元素？
* 为什么dict的键和set元素的顺序是根据它们被添加的次序而定的，以及为什么在映射对象的生命周期中，这个顺序并不是一成不变的？
* 为什么不应该在迭代循环dict或set的同时往里面添加元素？

### 3.9.1 一个关于效率的实验
如果在你的程序中有任何的磁盘输入/输出，那么不管查询有多少个元素的字典或集合，所耗费的时间都能忽略不计（前提是字典或集合不超过内存大小）。

但列表背后没有散列表支持`in`运算符，复杂度是`O(n)`。

### 3.9.2 字典中的散列表
散列表其实是一个稀疏数组（即总是有空白元素的数组）。散列表里的单元通常称为表元。在`dict`中，每个键值对要占用一个表元，每个表元包含两部分，一是对键的引用，二是对值的引用。因为所有表元的大小一致，可以通过偏移量来读取某个表元。

Python会设法保证大概还有1/3的表元是空的，所以在快要达到这个阈值的时候，原有的散列表会被复制到一个更大的空间里面。

如果要把一个对象放入散列表里，首先要计算这个元素的散列值，Python中使用`hash()`来做这件事。

#### 1. 散列值和相等性
内置的`hash()`方法可以用于所有内置类型对象。如果是自定义对象调用`hash()`，实际运行的是自定义的`__hash__`。如果两个对象比较时是相等的，那么它们的散列值必须相等，比如`1 == 1.0`为真，则`hash(1) == hash(1.0)为真`。

为了让散列值胜任散列表索引的角色，它们在散列空间必须尽尽量分散开来。这意味着最理想的是，越是相似但不相等的对象，它们的散列值应该差别越大。比如1和1.0散列值相同，但是和1.00001的散列值差别很大。

In [219]:
bin(hash(1)) # CPython：如果一个整型对象，能被存入一个机器字中，那么它的散列值就是它本身的值。

'0b1'

In [217]:
bin(hash(1.0))

'0b1'

In [218]:
bin(hash(1.00001))

'0b101001111100010110101100010001110010000000001'

> 从Python3.3开始, str, bytes和datetime对象的散列值计算过程多了随机“加盐”这一步。“盐值”是Python进程的一个常量，但每次启动Python解释器都会生成新的盐值。随机盐值加入的目的是防止DOS攻击。

#### 2. 散列值和相等性
`my_dict[search_key]`的处理过程：
* 首先调用`hash(search_key)`来计算散列值；
* 把散列值最低的几位数字当作偏移量，在散列表中查找表元（具体取几位，视当前散列表大小决定）；
* 若找到的表元是空的，则抛出`KeyError`异常；
* 若非空，表元内有一对`found_key: found_value`，Python会检查`search_key == found_key`是否为真，若真，返回`found_value`；若假，即散列冲突，则使用散列值的另一部分当作索引寻找表元。

添加新元素：发现空元时会放入新元素；\
更新现有键值：在找到对应表元后，用新值替换原表里的值对象。

插入新值时，Python会根据散列表的拥挤程度决定是否为其重新分配空间来扩容。如果散列表变大，那散列值所占位数和用作索引的位数都会增加，减少散列冲突发生的概率。

实际使用中，即使`dict`中有数百万个元素，散列冲突也很少发生。

### 3.9.3 `dict`的实现及其导致的后果
#### 1. 键必须是可散列的
一个可散列对象必须满足以下要求：
* 支持`hash()`函数，并且通过`__hash__`所得到的散列值是不变的；
* 支持通过`__eq__`方法检测相等性；
* 若`a == b`为真，则`hash(a) == hash(b)`也为真。

> 如果你实现了一个类的`__eq__`方法，并且希望它是可散列的，那么它一定要有个恰当的`__hash__`方法，保证上述第三条，否则会破坏恒定的散列表算法。另外，如果含有自定义`__eq__`的类处于可变的状态，就不要实现`__hash__`方法，因为它的实例是不可散列的。

#### 2. 字典在内存上开销巨大
原因是字典使用散列表，而散列表是稀疏的。

在存放数量巨大的记录时，使用元组或具名元组构成的列表会比字典构成的列表好：
* 避免散列表浪费的时间；
* 避免在每个元素中都存一遍记录中的字段名。

> * 在用户自定义的类型中，`__slots__`属性可以改变实例属性的存储方式，由`dict`变成`tuple`（见9.8节）。
> * 我们现在谈的是空间优化，但空间优化往往是可维护性的对立面，如果空间不紧张，可以先不考虑空间优化。

#### 3. 键查询很快
`dict`的实现是典型的空间换时间。

#### 4. 键的次序取决于添加顺序
原因是散列冲突时新键可能会被存放到另一个位置。

In [1]:
# 将同样的数据以不同的顺序添加到3个字典中
DIAL_CODES = [(86, 'China'), 
              (91, 'Inida'), 
              (1, 'United States'), 
              (62, 'Indonesia'), 
              (55, 'Brazil')]

d1 = dict(DIAL_CODES)
d2 = dict(sorted(DIAL_CODES))
d3 = dict(sorted(DIAL_CODES, key=lambda x: x[1]))

In [2]:
print(d1.keys())
print(d2.keys())
print(d3.keys())

dict_keys([86, 91, 1, 62, 55])
dict_keys([1, 55, 62, 86, 91])
dict_keys([55, 86, 62, 91, 1])


In [3]:
d1 == d2 == d3 # 虽然键的次数是乱的，这3个字典依然被认为是相等的

True

#### 5. 往字典里添加新键可能会改变已有键的顺序
添加新键时Python可能会为字典扩容，即新建一个大散列表，把字典中已有的元素添加到新表里，这个过程可能发生新的散列冲突，导致新散列表中键的次序变化。注意，以上变化是否发生以及如何发生，都依赖于字典背后的实现，所以我们无法确切知道背后发生了什么。

如果在迭代字典所有键的同时对字典进行修改，那么这个循环可能会跳过一些键——甚至是跳过字典中已有的键。

建议：首先迭代，在一个新字典中存放需要添加的内容；迭代结束后再对原字典进行修改。

> Python3中，`.keys()`，`.items()`和`.values()`返回的是字典视图（更像集合而非如Python2一般返回列表），视图是动态的。

### 3.9.4 `set`的实现以及导致的结果
`set`和`frozenset`的实现也依赖散列表。特点如前述`dict`的特点。

## 3.10 本章小结
字典算是Python的基石。除了`dict`，`collections`模块还提供了`defaultdict`，`ChainMap`和`Counter`等特殊映射类型，以及便于扩展的`UserDict`类。
 
 大多数映射类型都提供两个很强大的方法：`setdefault`和`update`：
 * `setdefault`可以更新字典里存放的可变值（比如列表），避免重复的键搜索；
 * `update`允许我们批量插入新值或更新已有键值对，其参数可以是键值对组成的迭代对象或映射对象（映射类型的构造方法会利用`update`方法）。
 
 
在映射类型的API中，有个很好用的方法是`__missing__`，可以通过自定义该方法决定找不到键发生什么。
 
`collections.abc`模块提供了`Mapping`和`MutableMapping`两个抽象基类，利用它们，我们可以进行类型查询或引用。`MappingProxyType`可以用来创建不可变映射类型。另外还有`Set`和`MutableSet`两个抽象基类。
 
 `dict`和`set`背后的散列表效率很高，理解其实现，我们会理解为什么被保存的元素会呈现不同的顺序，以及已有元素顺序发生变化的原因。同时，速度是牺牲空间换来的。